In [ ]:
import os
import torch
import torchvision
!pip install torchsummary
import torch.nn as nn
from torchvision import models
from torchsummary import summary

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# minibatch size
batch_size = 16

In [ ]:
from torchvision import transforms
from PIL import Image

# Define resize operation
resize = transforms.Resize((98, 98))

# Define transformations for both training and testing
transformations = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    resize,
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.RandomRotation(10),  # Data augmentation
    transforms.RandomResizedCrop((98, 98), scale=(0.8, 1.2), interpolation=Image.BILINEAR),  # Random crop
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),  # Normalizing for grayscale image
])

# Use the same transformations for both training and testing datasets
trainTransforms = transformations
testTransforms = transformations


In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split


# Load the dataset from the image folder
dataset = datasets.ImageFolder(root='/kaggle/input/fer2013/train', transform=trainTransforms)

# Calculate the sizes for train and validation sets
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size

# Split the dataset
train_data, validation_data = random_split(dataset, [train_size, val_size])
# Setup the batch size hyperparameter
BATCH_SIZE = batch_size
test_data = ImageFolder('/kaggle/input/fer2013/test', transform=testTransforms)

# Turn datasets into iterables (batches)
train_dataloader = DataLoader(train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

validation_dataloader = DataLoader(validation_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)


# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}")
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of Validation dataloader: {len(validation_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

In [ ]:
nb_train_samples = 1256 * 16
nb_validation_samples = 539 * 16
nb_test_samples = 449 * 16

In [ ]:
import torch
import torch.nn as nn

# Defining the neural network
class Designed_Model(nn.Module):
    def __init__(self):
        super(Designed_Model, self).__init__()
        
        # Convolutional blocks: Conv layer => BatchNorm => ReLU
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv_block5 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        
        # Adaptive average pooling
        self.adaptive_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(1024, 4096),  # Adjust the linear layer size based on your conv output
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 7),
        ) 
        
  

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)
        x = self.adaptive_avg_pool(x)
        x = x.view(x.size(0), -1)  
        x = self.fc_layers(x)
        return x

# Instantiating the model
model = Designed_Model()
print(model)

In [ ]:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch.nn.functional as F

criterion = nn.CrossEntropyLoss()
 
# Adam Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Number of epochs
num_epochs = 50


# Move model to GPU if available
model = model.to(device)

# Training Loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)  # Use KLD loss here
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_acc = running_corrects.double() / len(train_dataloader.dataset)
    print(f'Epoch {epoch}/{num_epochs - 1} - Training loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0
    
    with torch.no_grad():
        for inputs, labels in validation_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)  # Use KLD loss here
            _, preds = torch.max(outputs, 1)
            val_running_loss += loss.item() * inputs.size(0)
            val_running_corrects += torch.sum(preds == labels.data)

    val_epoch_loss = val_running_loss / len(validation_dataloader.dataset)
    val_epoch_acc = val_running_corrects.double() / len(validation_dataloader.dataset)
    print(f'Epoch {epoch}/{num_epochs - 1} - Validation loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_acc:.4f}')

    

# Save the model after training
torch.save(model.state_dict(), 'vggnet_CE_adam.pth')


In [ ]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
test_corrects = 0


nb_test_samples = len(test_dataloader.dataset)

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)

        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels.data).item()

# Calculate the average loss and accuracy
test_epoch_loss = test_loss / nb_test_samples
test_epoch_accuracy = test_corrects / nb_test_samples
test_accuracy_percentage = test_epoch_accuracy * 100

print(f'Test loss: {test_epoch_loss:.4f}')
print(f'Test Accuracy: {test_accuracy_percentage:.2f}%')
